In [72]:
from src.app.drivers import BaseDriver, OrdersPageDriver
import os
from dotenv import load_dotenv
import argparse
import pandas as pd
from datetime import datetime
import logging
from src.utils.log_config import setup_logger
from src.utils.order_handler import OrderHandler
from src.utils.data_merger import DataMerger
from src.utils.excel_formatter import ExcelFormatter


In [76]:
print(load_dotenv())

True


In [74]:
class Main:

    # ---------------------------------- Instance attributes ----------------------------------
    def __init__(self, headless=False):
        self.base_driver = BaseDriver(headless=headless)
        # self.os_type = self.base_driver.os_type
        self.os_type = 'Windows'
        self.orders_page_driver = OrdersPageDriver(self.base_driver)
        self.order_handler = OrderHandler()
        self.today = datetime.today().strftime('%m.%d.%y')
        self.excel_file_name = f'DD {self.today}.xlsx'
        self.json_build_file_path = ''
        # self.set_output_paths()  # set paths dependent on os type


    # ---------------------------------- Instance attributes ----------------------------------
    def set_paths_for_darwin(self):
        dev_excel_output_base_path = os.environ.get('DARWIN_DEV_EXCEL_PATH')
        print(f'Using basepath: {dev_excel_output_base_path}')
        if dev_excel_output_base_path is None:
            raise EnvironmentError("DARWIN_DEV_EXCEL_PATH environment variable is not set.")
            
        self.excel_output_file_path = os.path.join(dev_excel_output_base_path, self.excel_file_name)
        print(f"repr(self.excel_output_file_path is {repr(self.excel_output_file_path)}\n")
        print(f'\nUsing OS "{self.os_type}"\nSetting final Excel output filepath to: "{self.excel_output_file_path}"\n')
        
        self.json_build_file_path = os.environ.get('DARWIN_DEV_JSON_BUILDS_PATH')
        if self.json_build_file_path is None:
            raise EnvironmentError("DARWIN_DEV_JSON_BUILDS_PATH environment variable is not set.")
            
        print(f'Setting JSON CSV builds to: "{self.json_build_file_path}"\n')
    
    
    def set_paths_for_windows(self):
        stage_excel_output_base_path = os.environ.get('WINDOWS_STAGE_EXCEL_PATH')
        print(f'Using basepath: {stage_excel_output_base_path}')
        if stage_excel_output_base_path is None:
            raise EnvironmentError("WINDOWS_STAGE_EXCEL_PATH environment variable is not set.")
            
        # self.excel_output_file_path = os.path.join(stage_excel_output_base_path, self.excel_file_name)
        self.excel_output_file_path =  os.path.normpath(stage_excel_output_base_path, self.excel_file_name)
        print(f"repr(self.excel_output_file_path is {repr(self.excel_output_file_path)}\n")

        print(f'\nUsing OS: "{self.os_type}"\nSetting final Excel output filepath to: "{self.excel_output_file_path}"\n')
        
        self.json_build_file_path = os.environ.get('WINDOWS_STAGE_JSON_BUILDS_PATH')
        if self.json_build_file_path is None:
            raise EnvironmentError("WINDOWS_STAGE_JSON_BUILDS_PATH environment variable is not set.")
            
        print(f'Setting JSON CSV builds to: "{self.json_build_file_path}"\n')
    
    
    def set_output_paths(self):
        try:
            if self.os_type != 'Darwin':
                print(f'Not using Mac OS. Defaulting to STAGE env variables for filepaths....\n')
                self.set_paths_for_windows()
            else:
                self.set_paths_for_darwin()
    
        except EnvironmentError as e:
            print(f"Error: {e}")

            

    def switch_to_history_tab(self):
        switched_to_history_tab = self.orders_page_driver.switch_to_history_tab()
        if not switched_to_history_tab:
            logging.error('Could not orders_page_driver.switch_to_history_tab')
            return False
        else:
            logging.info('Successfully orders_page_driver.switch_to_history_tab')
            return True

    def set_date_filter_to_yesterday(self):
        date_filter_set_to_yesterday = self.orders_page_driver.set_date_filter_to_yesterday()
        if not date_filter_set_to_yesterday:
            logging.error('Could not orders_page_driver.set_date_filter_to_yesterday')
            return False
        else:
            logging.info('Successfully orders_page_driver.set_date_filter_to_yesterday')
            return True

    def switch_to_history_tab_and_set_date_filter_to_yesterday(self):
        try:

            switched_to_history_tab = self.switch_to_history_tab()
            if not switched_to_history_tab:
                logging.error(
                    'Could not switch_to_history_tab in switch_to_history_tab_and_set_date_filter_to_yesterday')
                return False, False

            date_filter_set_to_yesterday = self.set_date_filter_to_yesterday()
            if not date_filter_set_to_yesterday:
                logging.error(
                    'Could not set_date_filter_to_yesterday in switch_to_history_tab_and_set_date_filter_to_yesterday')
                return True, False

            if switched_to_history_tab and date_filter_set_to_yesterday:
                logging.info(f'Successfully switched_to_history_tab and date_filter_set_to_yesterday')
                return True, True

        except Exception as e:
            logging.exception(
                f'An error occurred attempting to switch_to_history_tab_and_set_date_filter_to_yesterday {e}')
            return False, False

    def get_sheet_name(self, order_df):
        try:
            store_num = order_df[0]['Store Number']
            sheet_name = f'#{store_num}'

            if sheet_name == '#NaN':
                logging.warning(
                    f"For OrderID: '{order_df[0]['Order']}' 'Store Number' could not be matched using Pick Up Location: '{order_df[0]['Pick Up Location']}'. Sheet name has been set to OrderID: '{order_df[0]['Order']}'.")
                sheet_name = order_df[0]['Order']
            return sheet_name, True
        except Exception as e:
            logging.exception(f"An error occurred while getting sheet name: {e}")
            return None, False

    def get_orders(self):
        try:
            orders = self.orders_page_driver.get_orders()
            if not orders:
                logging.error('Could not scrape orders data')
                return None
            else:
                return orders
        except Exception as e:
            logging.exception(f'An error occurred: {e}')
            return None

    def get_excel_output(self, orders_dfs):
        try:
            sheet_name_count = {}  # This dictionary will store the frequency of each sheet_name
            with pd.ExcelWriter(self.excel_output_file_path, engine='xlsxwriter') as writer:
                for idx, order_df in enumerate(orders_dfs):
                    sheet_name, success = self.get_sheet_name(order_df)
                    if not success:
                        return False

                    # Check if the sheet_name is already used
                    if sheet_name in sheet_name_count:
                        sheet_name_count[sheet_name] += 1  # Increment the frequency
                        suffix = chr(96 + sheet_name_count[
                            sheet_name])  # Convert frequency to alphabet (1 -> 'a', 2 -> 'b', ...)
                        sheet_name = f"{sheet_name}{suffix}"  # Append suffix to sheet_name
                    else:
                        sheet_name_count[sheet_name] = 1  # This is the first occurrence of this sheet_name

                    order_df.to_excel(writer, sheet_name, header=False, index=True)

                    formatter = ExcelFormatter(writer, sheet_name, order_df)
                    formatter.apply_sheet_formats()

            return True
        except Exception as e:
            logging.exception(f"An error occurred while generating Excel output: {e}")
            return False

    def run_main(self):
        self.setup()
        if not self.switch_to_history_tab_and_set_date_filter_to_yesterday():
            logging.error("Failed to initialize DoorDash Bot")
            return

        raw_orders = self.get_raw_orders()
        if raw_orders is None:
            logging.error("Failed to fetch orders")
            return

        orders_with_store_nums = self.merge_data(raw_orders)
        orders_dfs = self.convert_to_dataframes(orders_with_store_nums)
        print(f'\n*****************************************\n{orders_dfs}\n*****************************************\n')
        excel_output = self.export_to_excel(orders_dfs)
        if not excel_output:
            logging.error(f'Orders spreadsheet could not be exported.')
        else:
            logging.info(f'Orders Spreadsheet has been saved to: {self.excel_output_file_path}\nExiting....')
            self.base_driver.teardown_driver()

    def setup(self):
        setup_logger()

    def get_raw_orders(self):
        return self.get_orders()

    def merge_data(self, raw_orders):
        orders_json = self.order_handler.get_prettified_and_mapped_orders(raw_orders, is_raw=True)
        self.output_json(orders_json, 'orders_json.csv', 'Writing orders_json stdout...')

        data_merger = DataMerger(orders_json)
        return data_merger.add_store_numbers_to_orders()

    def output_json(self, json_str, filepath, log_message):
        output_filepath = os.path.join(self.json_build_file_path, filepath)
        self.order_handler.json_str_to_file(json_str=json_str, output_filepath=output_filepath, log_message=log_message)

    def convert_to_dataframes(self, orders_with_store_nums):
        orders_json_with_store_nums = self.order_handler.get_prettified_and_mapped_orders(orders_with_store_nums, is_raw=False)
        self.output_json(orders_json_with_store_nums, 'orders_json_with_store_nums.csv',
                         'Writing orders_json with store_num to stdout...')

        return self.order_handler.convert_flattened_orders_to_df(orders_with_store_nums)

    def export_to_excel(self, orders_dfs):
        excel_output = self.get_excel_output(orders_dfs)
        return excel_output


In [75]:
md = Main()
md.set_output_paths()

Initializing BaseDriver...
Using operating system: "Darwin".
Constructing chromedriver instance using executable_path: "/opt/homebrew/bin/chromedriver"
Not using Mac OS. Defaulting to STAGE env variables for filepaths....

Using basepath: C:\\Users\\ekima\\workspace\\TXB\\mock\\g-drive\\imports\\ir\\Door Dash\\DD Daily Order Details\\
repr(self.excel_output_file_path is 'C:\\\\Users\\\\ekima\\\\workspace\\\\TXB\\\\mock\\\\g-drive\\\\imports\\\\ir\\\\Door Dash\\\\DD Daily Order Details\\\\/DD 10.10.23.xlsx'


Using OS: "Windows"
Setting final Excel output filepath to: "C:\\Users\\ekima\\workspace\\TXB\\mock\\g-drive\\imports\\ir\\Door Dash\\DD Daily Order Details\\/DD 10.10.23.xlsx"

Setting JSON CSV builds to: "C:\\Users\\ekima\\workspace\\TXB\\dd-bot\\dev\\build"


In [15]:
load_dotenv()

Python-dotenv could not parse statement starting at line 3


True

In [28]:
load_dotenv()

True

In [29]:
test_path = os.environ.get('DARWIN_DEV_EXCEL_PATH')
print(test_path)
# /Users/ekim/workspace/txb/mock/g-drive/imports/ir/Door Dash/DD Daily Order Details/

/Users/ekim/workspace/txb/mock/g-drive/imports/ir/Door Dash/DD Daily Order Details/


In [30]:
test_path2 = os.environ.get('WINDOWS_STAGE_EXCEL_PATH')
print(test_path2)
# C:\\Users\\ekima\\workspace\\TXB\\mock\\g-drive\\imports\\ir\\Door Dash\\DD Daily Order Details\\

C:\\Users\\ekima\\workspace\\TXB\\mock\\g-drive\\imports\\ir\\Door Dash\\DD Daily Order Details\\


In [31]:
test_path3 = os.environ.get('WINDOWS_PROD_EXCEL_PATH')
print(test_path3)
# G:\Imports\IR\Door Dash\DD Daily Order Details\

G:\Imports\IR\Door Dash\DD Daily Order Details\


In [33]:
print(f"test_path2 is {repr(test_path2)}\n")
print(f"test_path3 is {repr(test_path3)}")


test_path2 is 'C:\\\\Users\\\\ekima\\\\workspace\\\\TXB\\\\mock\\\\g-drive\\\\imports\\\\ir\\\\Door Dash\\\\DD Daily Order Details\\\\'

test_path3 is 'G:\\Imports\\IR\\Door Dash\\DD Daily Order Details\\'
